In [11]:
from model.test import batch_acc, compute_loss
from data.generator import get_vocab_size, get_token2pos
import torch

In [105]:
loss = torch.nn.CrossEntropyLoss(reduction='none')

In [106]:
vocab_size = get_vocab_size()
BS = 16

In [190]:
targets = torch.zeros((BS, 4, vocab_size))

In [191]:
outputs = torch.randn((BS, 4, vocab_size))

In [192]:
token2pos=get_token2pos()

In [193]:
token2pos['2']

16

In [194]:
token2pos['0']

22

In [195]:
token2pos['#']

47

In [197]:
targets[:, 0, 16] = 1  # make all answers start with 2
targets[:, 1, 22] = 1  # make all answers 100
targets[:, 2, 22] = 1  # make all answers 100
targets[:, 3, 47] = 1  # add padding

In [198]:
outputs[:, 0, 41] += 100  # make all answers start with 1
outputs[:, 1, 22] += 100  # make all answers 100
outputs[:, 2, 22] += 100  # make all answers 100
outputs[:, 3, 2] += 100  # add other symbol instead of padding

In [199]:
outputs = [outputs[:, i, :].squeeze() for i in range(outputs.size(1))]

In [200]:
batch_acc(outputs, targets, vocab_size)

tensor(0.6667)

In [201]:
def compute_loss(loss, outputs, target):
    cumulative_loss = 0
    idx_pad = get_vocab_size() - 1  # get index padding
    idx_targets = target.argmax(dim=-1)  # make targets a matrix of indices
    mask = (idx_targets != idx_pad).type(torch.int32)  # compute integer mask
    for char_pos, output in enumerate(outputs):
        print(output.size(), target[:, char_pos, :].squeeze().size())
        print(torch.argmax(output, dim=1))
        print(torch.argmax(target[:, char_pos, :].squeeze(), dim=1))
        char_loss = loss(output, torch.argmax(target[:, char_pos, :].squeeze(), dim=1))  # compute loss for timestep in batch
        print(char_loss)
        masked_char_loss = char_loss * mask[:, char_pos]
        print(masked_char_loss)
        cumulative_loss += masked_char_loss.sum()
        print(cumulative_loss)
        print()
    avg_loss = cumulative_loss / mask.sum()
    return avg_loss

In [202]:
compute_loss(loss, outputs, targets)

torch.Size([16, 48]) torch.Size([16, 48])
tensor([41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41])
tensor([16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16])
tensor([ 98.6602, 100.1316, 101.1045,  98.5103, 100.3376, 102.6636, 101.0573,
        100.2924, 102.0731,  98.6486, 100.0426,  98.3982,  98.0179,  99.5142,
        101.0232,  98.8062])
tensor([ 98.6602, 100.1316, 101.1045,  98.5103, 100.3376, 102.6636, 101.0573,
        100.2924, 102.0731,  98.6486, 100.0426,  98.3982,  98.0179,  99.5142,
        101.0232,  98.8062])
tensor(1599.2815)

torch.Size([16, 48]) torch.Size([16, 48])
tensor([22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22])
tensor([22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22])
tensor([-0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.])
tensor([-0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.])
tensor(1599.2815)

torch.Size([16, 48]) torch.Size([16, 4

tensor(33.3184)

In [159]:
import numpy as np

In [187]:
np.random.randint(10**2)

27